In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor,export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error,accuracy_score,r2_score
import xgboost as xgb

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')
df.T

0                  1                  2     \
engine_displacement              170                130                170   
num_cylinders                    3.0                5.0                NaN   
horsepower                     159.0               97.0               78.0   
vehicle_weight           3413.433759        3149.664934        3079.038997   
acceleration                    17.7               17.8               15.1   
model_year                      2003               2007               2018   
origin                        Europe                USA             Europe   
fuel_type                   Gasoline           Gasoline           Gasoline   
drivetrain           All-wheel drive  Front-wheel drive  Front-wheel drive   
num_doors                        0.0                0.0                0.0   
fuel_efficiency_mpg        13.231729          13.688217          14.246341   

                                3                4                5     \
engine_displacement              220              210              190   
num_cylinders                    4.0              1.0              3.0   
horsepower                       NaN            140.0              NaN   
vehicle_weight           2542.392402       3460.87099      2484.883986   
acceleration                    20.2             14.4             14.7   
model_year                      2009             2009             2008   
origin                           USA           Europe           Europe   
fuel_type                     Diesel         Gasoline         Gasoline   
drivetrain           All-wheel drive  All-wheel drive  All-wheel drive   
num_doors                        2.0              2.0             -1.0   
fuel_efficiency_mpg        16.912736        12.488369        17.271818   

                                  6                7                  8     \
engine_displacement                240              150                250   
num_cylinders                      7.0              4.0                1.0   
horsepower                       127.0            239.0              174.0   
vehicle_weight             3006.542287       3638.65778         2714.21931   
acceleration                      22.2             17.3               10.3   
model_year                        2012             2020               2016   
origin                             USA              USA               Asia   
fuel_type                     Gasoline           Diesel             Diesel   
drivetrain           Front-wheel drive  All-wheel drive  Front-wheel drive   
num_doors                          1.0              1.0               -1.0   
fuel_efficiency_mpg          13.210412        12.848884          16.823554   

                                  9     ...               9694  \
engine_displacement                150  ...                220   
num_cylinders                      4.0  ...                3.0   
horsepower                       123.0  ...              176.0   
vehicle_weight             3509.036569  ...        2837.652898   
acceleration                      10.2  ...               11.4   
model_year                        2005  ...               2013   
origin                             USA  ...             Europe   
fuel_type                     Gasoline  ...           Gasoline   
drivetrain           Front-wheel drive  ...  Front-wheel drive   
num_doors                         -1.0  ...               -1.0   
fuel_efficiency_mpg          12.298355  ...          16.233553   

                                  9695             9696               9697  \
engine_displacement                280              190                220   
num_cylinders                      5.0              7.0                5.0   
horsepower                       154.0             69.0              110.0   
vehicle_weight             3797.309878       3133.32921         2532.98763   
acceleration                      16.6             16.6               14.3   
model_ye

In [5]:
categorical_columns = list(df.dtypes[df.dtypes == object].index)
numerical_columns = list(df.dtypes[df.dtypes !=object].index)
print(numerical_columns)
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(" ","_")

for col in numerical_columns:
    df[col] = df[col].fillna(0)

df_fulltrain,df_test=train_test_split(df,test_size=0.2,random_state=1)
df_train,df_val=train_test_split(df_fulltrain,test_size=0.25,random_state=1)
df_train= df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_train=df_train['fuel_efficiency_mpg'].values
y_val =df_val.fuel_efficiency_mpg.values
y_test = df_test.fuel_efficiency_mpg.values
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']
df_train


['engine_displacement', 'num_cylinders', 'horsepower', 'vehicle_weight', 'acceleration', 'model_year', 'num_doors', 'fuel_efficiency_mpg']


engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                     120            5.0       169.0     2966.679505   
1                     200            3.0       143.0     2950.822121   
2                     180            6.0       180.0     3078.221669   
3                     280            5.0       174.0     2797.991793   
4                     250            4.0       133.0     2362.426930   
...                   ...            ...         ...             ...   
5817                  230            3.0       176.0     3430.993044   
5818                  250            4.0       180.0     3067.664350   
5819                  230            2.0       182.0     3041.964593   
5820                  180            7.0       147.0     2453.341430   
5821                  190            2.0       138.0     2833.899434   

      acceleration  model_year  origin fuel_type         drivetrain  num_doors  
0             13.9        2005     usa  gasoline  front-wheel_drive       -1.0  
1             17.1        2013    asia    diesel  front-wheel_drive       -1.0  
2             17.4        2007     usa  gasoline    all-wheel_drive        0.0  
3              0.0        2016     usa    diesel    all-wheel_drive        0.0  
4             16.3        2010     usa    diesel  front-wheel_drive       -1.0  
...            ...         ...     ...       ...                ...        ...  
5817          17.9        2022  europe    diesel    all-wheel_drive        0.0  
5818          15.7        2010    asia    diesel    all-wheel_drive       -1.0  
5819          16.7        2010  europe    diesel    all-wheel_drive        0.0  
5820          15.2        2015  europe  gasoline    all-wheel_drive        0.0  
5821          14.1        2017     usa    diesel  front-wheel_drive        0.0  

[5822 rows x 10 columns]

In [6]:
dv= DictVectorizer(sparse=True)
train_dict = df_train.to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient = 'records')
X_val = dv.transform(val_dict)


In [21]:
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)
#print(export_text(dt,feature_names =list(dv.get_feature_names_out())))
dv.get_feature_names_out()[np.argmax(dt.feature_importances_)]

'vehicle_weight'

In [ ]:
n=10
rf_dt =RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=-1,verbose=2)
rf_dt.fit(X_train,y_train)
y_pred = rf_dt.predict(X_val)


print("RMSE for a Random forestRegressor with %f estimator =%f " %(n , root_mean_squared_error(y_val,y_pred)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 1 of 10
building tree 5 of 10building tree 6 of 10

building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
RMSE for a Random forestRegressor with 10.000000 estimator =0.459578 


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


To prepare a dataframe for a single vehicle entry  

In [19]:
for max_depth in [10,15,20,25]:
   scores =[]
   for n in range(10,210,10):
      rf_dt =RandomForestRegressor(n_estimators=n,random_state=1,n_jobs=-1,verbose=0,max_depth=max_depth)
      rf_dt.fit(X_train,y_train)
      y_pred = rf_dt.predict(X_val)
      scores.append(root_mean_squared_error(y_val,y_pred))
   print("RMSE for a Random forestRegressor with max_depth=%d is %.3f" %(max_depth, np.mean(scores)))

RMSE for a Random forestRegressor with max_depth=10 is 0.442
RMSE for a Random forestRegressor with max_depth=15 is 0.445
RMSE for a Random forestRegressor with max_depth=20 is 0.446
RMSE for a Random forestRegressor with max_depth=25 is 0.446


In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

Train the model with these parameters:
n_estimators=10,
max_depth=20,
random_state=1,
n_jobs=-1 (optional)
Get the feature importance information from this model

In [22]:
rf_dt=RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=-1,verbose=0,max_depth=20)
rf_dt.fit(X_train,y_train)
dv.get_feature_names_out()[np.argmax(rf_dt.feature_importances_)]

'vehicle_weight'

In [ ]:
features =list(dv.get_feature_names_out())
dtrain =xgb.DMatrix(X_train,label=y_train,feature_names=features)
dval = xgb.DMatrix(X_val,label=y_val,feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'eval')]
xgb_params= {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(params = xgb_params,dtrain = dtrain,num_boost_round=100,evals=watchlist)
y_pred = model.predict(dval)
#print(y_pred)
#print(accuracy_score(y_val, y_pred))
print("XGB RMSE for eta =%f: is %.3f" % (xgb_params['eta'], root_mean_squared_error(y_val,y_pred)))

[0]	train-rmse:2.40931	eval-rmse:2.46950


[1]	train-rmse:2.29536	eval-rmse:2.35198
[2]	train-rmse:2.18725	eval-rmse:2.24090
[3]	train-rmse:2.08454	eval-rmse:2.13540
[4]	train-rmse:1.98719	eval-rmse:2.03545
[5]	train-rmse:1.89481	eval-rmse:1.94098
[6]	train-rmse:1.80728	eval-rmse:1.85110
[7]	train-rmse:1.72418	eval-rmse:1.76624
[8]	train-rmse:1.64542	eval-rmse:1.68580
[9]	train-rmse:1.57073	eval-rmse:1.61007
[10]	train-rmse:1.49989	eval-rmse:1.53767
[11]	train-rmse:1.43287	eval-rmse:1.46996
[12]	train-rmse:1.36925	eval-rmse:1.40515
[13]	train-rmse:1.30909	eval-rmse:1.34435
[14]	train-rmse:1.25220	eval-rmse:1.28599
[15]	train-rmse:1.19829	eval-rmse:1.23118
[16]	train-rmse:1.14725	eval-rmse:1.17944
[17]	train-rmse:1.09902	eval-rmse:1.13085
[18]	train-rmse:1.05334	eval-rmse:1.08482
[19]	train-rmse:1.01029	eval-rmse:1.04113
[20]	train-rmse:0.96955	eval-rmse:1.00028
[21]	train-rmse:0.93116	eval-rmse:0.96157
[22]	train-rmse:0.89493	eval-rmse:0.92514
[23]	train-rmse:0.86073	eval-rmse:0.89064
[24]	train-rmse:0.82845	eval-rmse:0.85859
[